# Tesla Stock Price Forecasting: LSTM vs RNN Comparison

This notebook implements a comparative analysis of Long Short-Term Memory (LSTM) networks and Simple Recurrent Neural Networks (RNN) for predicting Tesla stock prices. The analysis includes data preprocessing, feature engineering, model training, evaluation, and forecasting.

The goal is to determine which model architecture performs better for this specific time series forecasting task and generate 30-day forecasts for Tesla stock.

## Libraries and Dependencies

This analysis uses the following libraries:

- **pandas & numpy**: For data manipulation and numerical operations
- **seaborn & matplotlib**: For data visualization and plotting
- **sklearn**: For data preprocessing, model evaluation metrics, and train-test splitting
- **tensorflow.keras**: For building and training deep learning models
- **time**: For measuring and comparing model training times
- **matplotlib.dates**: For handling date formatting in plots

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, SimpleRNN, Bidirectional
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import matplotlib.dates as mdates
import random
import os

## Data Preparation

The data preparation process includes:

1. **Loading the dataset**: Reading Tesla stock price data from CSV
2. **Preprocessing**: Converting data types, handling dates, and setting up the index
3. **Feature engineering**: 
   - Computing percentage changes in closing price
   - Calculating moving averages
4. **Sequence creation**: Converting time series data into supervised learning format
   - Each sequence of length 120 days becomes an input (X)
   - The next day becomes the target (y)
5. **Scaling**: Normalizing the data using MinMaxScaler to improve model training
6. **Train-test-validation split**: Dividing data into training, validation, and test sets

In [ ]:
# -------------- Set Random Seeds --------------
seed = 42
def set_seeds(seed_value=seed):
    """Set seeds for reproducibility across all libraries"""
    # Python built-in random module
    random.seed(seed_value)
    
    # NumPy
    np.random.seed(seed_value)
    
    # TensorFlow
    tf.random.set_seed(seed_value)
    
    # Set Python hash seed
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    # Configure TensorFlow for deterministic operations
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    
    # Configure TensorFlow session
    try:
        tf.config.experimental.enable_op_determinism()
    except:
        # Older TensorFlow versions might not have this
        print("TensorFlow enable_op_determinism not available in this version")
    
    # For TensorFlow 2.x compatibility - don't use set_session which is deprecated
    if hasattr(tf, 'compat') and hasattr(tf.compat, 'v1'):
        # Try to set up the TF 1.x style configs (for older TF 2.x versions)
        try:
            tf.compat.v1.set_random_seed(seed_value)
            session_conf = tf.compat.v1.ConfigProto(
                intra_op_parallelism_threads=1,
                inter_op_parallelism_threads=1
            )
            # Only set session if the function exists
            if hasattr(tf.compat.v1.keras.backend, 'set_session'):
                sess = tf.compat.v1.Session(config=session_conf)
                tf.compat.v1.keras.backend.set_session(sess)
        except Exception as e:
            print(f"TF 1.x compatibility mode setting failed: {e}")
    
    print(f"All seeds set to {seed_value} for reproducibility")

# Set seeds at the very beginning
# set_seeds(seed)


# -------------- Data Loading Functions --------------
def load_data(file_path):
    """Load and preprocess the dataset"""
    # Mount Google Drive if in Colab
    try:
        from google.colab import drive
        drive.mount("/content/drive", force_remount=True)
        print("Google Drive mounted successfully")
    except:
        print("Not running in Google Colab or drive already mounted")

    # Load data
    df = pd.read_csv(file_path)
    df = df.drop([0, 1], axis=0).reset_index(drop=True)

    # Convert columns to float
    for col in df.columns[1:]:
        df[col] = df[col].apply(lambda x: float(x))

    # Rename columns to lowercase
    df.columns = df.columns.str.lower()

    # Fix date column
    df['date'] = pd.to_datetime(df['price'])
    df.drop(['price'], inplace=True, axis=1)

    # Set date as index
    df.set_index('date', inplace=True)

    return df

# -------------- Feature Engineering --------------
def calculate_pct_change(df):
    """Calculate percentage change in closing price"""
    pct_change = []
    pct_change.append(0)
    for i in range(1, df.shape[0]):
        value = (df['close'][i] - df['close'][i-1])*100/df['close'][i-1]
        pct_change.append(value)

    return np.array(pct_change)

def calculate_moving_average(data, n_steps=15):
    """Calculate moving average of a time series"""
    ma = [0] * n_steps
    for i in range(n_steps, data.shape[0]):
        ma_val = np.mean(data.iloc[i - n_steps + 1: i+1])
        ma.append(ma_val)
    return np.array(ma)


# Function to calculate RSI
def calculate_rsi(data, window=14):
    """Calculate Relative Strength Index (RSI)"""
    # Calculate price changes
    delta = data.diff()
    
    # Create gain (up) and loss (down) series
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    
    # Calculate average gain and average loss
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    
    # Calculate relative strength
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Fill NaN values
    rsi[:window] = 50  # Fill initial NaN values with neutral RSI
    
    return rsi


# -------------- Sequence Creation --------------
def create_sequences(data, seq_length):
    """Create sequences for LSTM model"""
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Shape = (batch size, sequence length, features)
        y.append(data[i+seq_length])    # Next row as prediction target
    return np.array(X), np.array(y)


## Model Architecture

### LSTM Model

Long Short-Term Memory networks are specialized RNNs designed to capture long-term dependencies in sequential data:


In [ ]:

# -------------- Model Building --------------
def build_lstm_model(input_shape, units=100):
    """Build LSTM model architecture"""
    set_seeds(seed)
    model = Sequential()
    model.add(Input(shape=input_shape))
    # model.add((LSTM(units=units, return_sequences=True)))
    # model.add(Dropout(0.2))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=input_shape[1]))

    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model

def build_rnn_model(input_shape, units=100):
    """Build Vanilla RNN model architecture with the same structure as LSTM"""
    set_seeds(seed)
    model = Sequential()
    model.add(Input(shape=input_shape))
    # model.add(SimpleRNN(units=units, return_sequences=True))
    # model.add(Dropout(0.2))
    model.add(SimpleRNN(units=units, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=input_shape[1]))

    model.compile(optimizer='Adam', loss='mean_squared_error')
    return model


### 5. Forecasting Methodology

```markdown
## Forecasting Methodology

This analysis implements recursive multi-step forecasting:

1. Start with the last known sequence of actual data (120 days)
2. Make a prediction for the next day (t+1)
3. Shift the window forward by removing the oldest day (t-120)
4. Add the new prediction to the end of the window
5. Use this updated window to predict the next day (t+2)
6. Repeat steps 3-5 for the desired forecast horizon (30 days)

This approach allows for generating predictions beyond the available data, but can suffer from error accumulation over longer horizons as each prediction builds on previous predictions.

In [ ]:
# -------------- Forecasting Function --------------
def forecast_future(model, last_sequence, scaler, days_to_predict, seq_length, feature_count, start_date):
    """Generate future forecasts based on the model"""
    # Create array to store predictions
    forecasted_values = np.empty((days_to_predict, feature_count))

    # Create window for recursive prediction
    window = np.empty((days_to_predict, seq_length, feature_count))

    # Initial window from last sequence of actual data
    window[0, :, :] = last_sequence

    # First prediction
    to_predict = window[0, :, :].reshape(1, seq_length, feature_count)
    forecasted_values[0, :] = model.predict(to_predict, verbose=False)

    # Subsequent predictions
    for i in range(1, days_to_predict):
        window[i, :-1, :] = window[i-1, 1:, :]  # Remove first row of previous window
        window[i, -1, :] = forecasted_values[i-1, :]  # Add previous prediction
        to_predict = window[i, :, :].reshape(1, seq_length, feature_count)
        forecasted_values[i, :] = model.predict(to_predict, verbose=False)

    # Inverse transform scaled predictions
    forecasted_values = scaler.inverse_transform(forecasted_values)

    # Create DataFrame with date index
    forecast_index = pd.date_range(start=start_date, periods=days_to_predict, freq='B')
    forecasted_df = pd.DataFrame(forecasted_values, index=forecast_index)

    return forecasted_df

## Evaluation Metrics

The models are evaluated using multiple performance metrics:

- **Mean Squared Error (MSE)**: Average of squared differences between predictions and actual values
- **Root Mean Squared Error (RMSE)**: Square root of MSE, providing error in the same units as the target variable
- **Mean Absolute Error (MAE)**: Average of absolute differences between predictions and actual values
- **R² Score**: Proportion of variance in the dependent variable explained by the model
- **Mean Absolute Percentage Error (MAPE)**: Average percentage difference between predictions and actual values

These metrics provide a comprehensive view of model performance from different perspectives.

In [ ]:


# -------------- Performance Metrics --------------
def calculate_metrics(y_true, y_pred):
    """Calculate various performance metrics"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    # Calculate MAPE (Mean Absolute Percentage Error)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'MAPE (%)': mape
    }

## Visualization Functions

Multiple visualization functions help analyze and compare the models:

1. **Data Exploration**:
   - `plot_percentage_change()`: Visualizes daily price changes and their distribution
   - `plot_stock_with_ma()`: Shows stock price with moving average and volume
   - `plot_volatility_analysis()`: Examines price volatility over time
   - `plot_candlestick_chart()`: Traditional OHLC financial chart

2. **Model Comparison**:
   - `plot_model_comparison()`: Compares predictions against actual prices
   - `plot_error_comparison()`: Analyzes prediction errors for both models
   - `plot_close_predictions_detail()`: Detailed view of recent predictions
   - `plot_forecast_comparison()`: Compares future forecasts from both models
   - `plot_training_history()`: Visualizes training and validation loss curves
   - `compare_model_metrics()`: Side-by-side comparison of performance metrics

In [ ]:


# -------------- Visualization Functions --------------
def plot_percentage_change(df_index, pct_change):
    """Plot percentage change in closing price"""
    plt.figure(figsize=(20, 6))
    sns.lineplot(x=df_index, y=pct_change)
    plt.xlabel('Year')
    plt.ylabel('Percentage Change (%)')
    plt.title('Percentage Change in Tesla Stock Price')
    plt.grid(True)
    plt.show()

    # Plot distribution of percentage changes
    plt.figure(figsize=(12, 6))
    sns.histplot(x=pct_change, bins=100, kde=True)
    plt.title(f"Distribution of Percentage Changes (Mean: {pct_change.mean():.4f}%)")
    plt.axvline(pct_change.mean(), color='r', linestyle='--')
    plt.show()

def plot_stock_with_ma(df, ma, n_steps):
    """Plot stock price with moving average and volume"""
    plt.figure(figsize=(20, 6))
    plt.plot(df.index, df['close'], 'b-', label='Close Price')
    plt.bar(df.index, df['volume']/3e6, alpha=0.3, label='Volume (scaled)')
    plt.plot(df.index[n_steps:], ma[n_steps:], 'r', label=f'{n_steps}-day Moving Avg')
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.title("Tesla Stock Price with Moving Average and Volume")
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_model_comparison(df, lstm_pred, rnn_pred, lstm_forecast=None, rnn_forecast=None):
    """Plot LSTM vs RNN predictions and forecasts"""
    plt.figure(figsize=(20, 10))

    # Plot actual values
    plt.plot(df.index, df['close'], 'k-', label='Actual', linewidth=2)

    # Plot LSTM and RNN predictions
    plt.plot(lstm_pred.index, lstm_pred['close'], 'b-', label='LSTM Predicted', alpha=0.8)
    plt.plot(rnn_pred.index, rnn_pred['close'], 'g-', label='RNN Predicted', alpha=0.8)

    # Plot forecasts if provided
    if lstm_forecast is not None and rnn_forecast is not None:
        plt.axvline(x=lstm_forecast.index[0], color='k', linestyle='--', label='Forecast Start')
        plt.plot(lstm_forecast.index, lstm_forecast['close'], 'b--', label='LSTM Forecast')
        plt.plot(rnn_forecast.index, rnn_forecast['close'], 'g--', label='RNN Forecast')

    plt.grid(True)
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.title('Tesla Stock Price: LSTM vs RNN Comparison')
    plt.legend()
    plt.show()

def plot_error_comparison(df, lstm_pred, rnn_pred):
    """Plot error comparison between LSTM and RNN"""
    # Calculate errors
    lstm_error = lstm_pred['close'] - df.loc[lstm_pred.index, 'close']
    rnn_error = rnn_pred['close'] - df.loc[rnn_pred.index, 'close']

    plt.figure(figsize=(20, 12))

    # Plot 1: Absolute errors
    plt.subplot(2, 1, 1)
    plt.plot(lstm_pred.index, np.abs(lstm_error), 'b-', label='LSTM |Error|')
    plt.plot(rnn_pred.index, np.abs(rnn_error), 'g-', label='RNN |Error|')
    plt.title('Absolute Prediction Errors: LSTM vs RNN')
    plt.ylabel('Absolute Error ($)')
    plt.legend()
    plt.grid(True)

    # Plot 2: Error distributions
    plt.subplot(2, 1, 2)
    plt.hist(lstm_error, bins=50, alpha=0.5, label='LSTM Error', color='blue')
    plt.hist(rnn_error, bins=50, alpha=0.5, label='RNN Error', color='green')
    plt.axvline(lstm_error.mean(), color='blue', linestyle='--', label=f'LSTM Mean Error: {lstm_error.mean():.2f}')
    plt.axvline(rnn_error.mean(), color='green', linestyle='--', label=f'RNN Mean Error: {rnn_error.mean():.2f}')
    plt.title('Error Distribution: LSTM vs RNN')
    plt.xlabel('Error ($)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

def plot_close_predictions_detail(df, lstm_pred, rnn_pred, window_size=30):
    """Plot detailed close price predictions for the last window_size days"""
    # Get the last window_size days
    last_days = df.index[-window_size:]

    # Filter predictions for those days
    lstm_last = lstm_pred.loc[lstm_pred.index.isin(last_days)]
    rnn_last = rnn_pred.loc[rnn_pred.index.isin(last_days)]
    actual_last = df.loc[last_days]

    plt.figure(figsize=(20, 8))
    plt.plot(actual_last.index, actual_last['close'], 'ko-', label='Actual', linewidth=2)
    plt.plot(lstm_last.index, lstm_last['close'], 'bs-', label='LSTM', markersize=8)
    plt.plot(rnn_last.index, rnn_last['close'], 'g^-', label='RNN', markersize=8)

    plt.grid(True)
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.title(f'Detailed Comparison of Last {window_size} Days Predictions')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_forecast_comparison(lstm_forecast, rnn_forecast):
    """Plot detailed comparison of LSTM and RNN forecasts"""
    plt.figure(figsize=(20, 8))

    plt.plot(lstm_forecast.index, lstm_forecast['close'], 'b-o', label='LSTM Forecast')
    plt.plot(rnn_forecast.index, rnn_forecast['close'], 'g-^', label='RNN Forecast')

    # Calculate the difference between forecasts
    diff = lstm_forecast['close'] - rnn_forecast['close']

    # Plot the difference as a bar chart on a secondary axis
    ax1 = plt.gca()
    ax2 = ax1.twinx()
    ax2.bar(lstm_forecast.index, diff, alpha=0.3, color='r', label='LSTM - RNN')
    ax2.set_ylabel('Difference ($)')

    # Add grid and labels
    plt.grid(True)
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price ($)')
    plt.title('Forecast Comparison: LSTM vs RNN')

    # Combine legends from both axes
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='best')

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def plot_training_history(lstm_history, rnn_history):
    """Plot training and validation loss history for both models"""
    plt.figure(figsize=(15, 10))

    # Plot training losses
    plt.subplot(2, 1, 1)
    plt.plot(lstm_history.history['loss'], label='LSTM Training Loss')
    plt.plot(rnn_history.history['loss'], label='RNN Training Loss')
    plt.title('Training Loss: LSTM vs RNN')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')  # Log scale often helps visualize the convergence better

    # Plot validation losses
    plt.subplot(2, 1, 2)
    plt.plot(lstm_history.history['val_loss'], label='LSTM Validation Loss')
    plt.plot(rnn_history.history['val_loss'], label='RNN Validation Loss')
    plt.title('Validation Loss: LSTM vs RNN')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')

    plt.tight_layout()
    plt.show()

def plot_individual_training_history(lstm_history, rnn_history):
    """Plot separate training and validation loss charts for each model"""
    # LSTM Loss Plot
    plt.figure(figsize=(15, 6))
    plt.plot(lstm_history.history['loss'], label='Training Loss')
    plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
    plt.title('LSTM Model: Training vs Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')
    plt.tight_layout()
    plt.show()

    # RNN Loss Plot
    plt.figure(figsize=(15, 6))
    plt.plot(rnn_history.history['loss'], label='Training Loss')
    plt.plot(rnn_history.history['val_loss'], label='Validation Loss')
    plt.title('RNN Model: Training vs Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')
    plt.tight_layout()
    plt.show()

def plot_volatility_analysis(df):
    """Plot volatility analysis over time"""
    # Calculate rolling standard deviation (volatility)
    volatility = df['close'].rolling(window=20).std()

    plt.figure(figsize=(20, 10))

    # Plot 1: Stock Price
    plt.subplot(2, 1, 1)
    plt.plot(df.index, df['close'], label='Close Price')
    plt.title('Tesla Stock Price')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True)

    # Plot 2: Volatility
    plt.subplot(2, 1, 2)
    plt.plot(df.index, volatility, color='red', label='20-day Volatility')
    plt.title('Tesla Stock Price Volatility (20-day Rolling Std)')
    plt.xlabel('Date')
    plt.ylabel('Volatility ($)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

def plot_candlestick_chart(df, days=90):
    """Plot candlestick chart for the last n days"""
    # Get last n days of data
    data = df.tail(days)

    fig, ax = plt.subplots(figsize=(20, 10))

    # Calculate the distance between points for width of candlesticks
    width = 0.6

    # Create candlestick colors
    up = data[data.close >= data.open]
    down = data[data.close < data.open]

    # Plot candlesticks
    ax.bar(up.index, up.close-up.open, width, bottom=up.open, color='green', alpha=0.5)
    ax.bar(down.index, down.close-down.open, width, bottom=down.open, color='red', alpha=0.5)

    # Plot high-low lines
    ax.vlines(up.index, up.low, up.high, color='green', linewidth=1)
    ax.vlines(down.index, down.low, down.high, color='red', linewidth=1)

    plt.title(f'Tesla Stock - Candlestick Chart (Last {days} Trading Days)')
    plt.ylabel('Price ($)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def compare_model_metrics(metrics_lstm, metrics_rnn):
    """Compare model metrics side by side"""
    metrics = pd.DataFrame({
        'LSTM': metrics_lstm,
        'RNN': metrics_rnn
    })

    plt.figure(figsize=(12, 8))

    # Plot the metrics as a bar chart
    ax = metrics.plot(kind='bar', color=['blue', 'green'], alpha=0.7)

    # Add value labels on top of bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.4f')

    plt.title('Model Performance Metrics: LSTM vs RNN')
    plt.ylabel('Value')
    plt.xlabel('Metric')
    plt.grid(axis='y')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Print the metrics table
    print("\n--- Model Performance Comparison ---")
    print(metrics)

    # Calculate improvement percentages
    improvement = ((metrics['RNN'] - metrics['LSTM']) / metrics['RNN'] * 100).abs()
    better_model = ['LSTM' if metrics['LSTM'][i] < metrics['RNN'][i] else 'RNN' for i in range(len(metrics))]

    # For R², higher is better
    if 'R²' in metrics.index:
        idx = metrics.index.get_loc('R²')
        better_model[idx] = 'LSTM' if metrics['LSTM'][idx] > metrics['RNN'][idx] else 'RNN'

    comparison = pd.DataFrame({
        'LSTM': metrics['LSTM'],
        'RNN': metrics['RNN'],
        'Improvement (%)': improvement,
        'Better Model': better_model
    })

    print("\n--- Detailed Comparison ---")
    print(comparison)


def plot_stock_with_ma_rsi(df, ma, n_steps, rsi):
    """Plot stock price with moving average, volume and RSI"""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), gridspec_kw={'height_ratios': [3, 1]})
    
    # Top plot: Price, Volume, and MA
    ax1.plot(df.index, df['close'], 'b-', label='Close Price')
    ax1.bar(df.index, df['volume']/3e6, alpha=0.3, label='Volume (scaled)')
    ax1.plot(df.index[n_steps:], ma[n_steps:], 'r', label=f'{n_steps}-day Moving Avg')
    ax1.set_ylabel("Price ($)")
    ax1.set_title("Tesla Stock Price with Moving Average, Volume, and RSI")
    ax1.legend(loc='upper left')
    ax1.grid(True)
    
    # Bottom plot: RSI
    ax2.plot(df.index, rsi, 'g-', label='RSI')
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
    ax2.set_ylim(0, 100)
    ax2.fill_between(df.index, rsi, 70, where=(rsi>=70), color='r', alpha=0.3)
    ax2.fill_between(df.index, rsi, 30, where=(rsi<=30), color='g', alpha=0.3)
    ax2.set_ylabel("RSI")
    ax2.set_xlabel("Date")
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()



## Results and Analysis

The analysis compares LSTM and RNN models based on:

1. **Training Performance**:
   - Training time
   - Training and validation loss curves
   - Convergence behavior

2. **Prediction Accuracy**:
   - Test set prediction errors
   - Performance on various metrics (MSE, RMSE, MAE, R², MAPE)
   - Error distribution analysis

3. **Forecasting**:
   - 30-day price forecasts
   - Forecast divergence between models
   - Percentage change from last known price

The results help determine which architecture is better suited for Tesla stock price prediction and whether the additional complexity of LSTM provides meaningful improvements over simple RNN.

In [ ]:
# -------------- Main Execution --------------
def main():
    # Configuration parameters
    set_seeds(seed)
    file_path = "/kaggle/input/tesla-stock-price-data-2000-2025/tesla_stock_data_2000_2025.csv"
    seq_length = 180
    units = 20
    batch_size = 32
    epochs = 25
    days_to_predict = 30
    test_size = 0.15
    validation_size = 0.15  
    n_steps_ma = 10

    # Load data
    df = load_data(file_path)
    print("Data loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(df.head())

    # Data exploration and visualization
    print("\n--- Data Statistics ---")
    print(df.drop(['date'], axis=1, errors='ignore').describe())

    # Calculate and plot percentage change
    pct_change = calculate_pct_change(df)
    plot_percentage_change(df.index, pct_change)

    # Calculate RSI
    rsi = calculate_rsi(df['close'], window=14)

    # Calculate and plot moving average
    ma = calculate_moving_average(df['close'], n_steps=n_steps_ma)
    # plot_stock_with_ma(df, ma, n_steps_ma)
    # Plot stock with MA and RSI
    plot_stock_with_ma_rsi(df, ma, n_steps_ma, rsi)

    # Additional visualizations
    plot_volatility_analysis(df)
    plot_candlestick_chart(df, days=90)

    # Data preparation for model
    scaler = MinMaxScaler()
    # df = df.iloc[1700:, :]
    data = scaler.fit_transform(df.iloc[:, :-1].values)

    # Create sequences
    X, y = create_sequences(data, seq_length)
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    # Split into train, validation, and test sets
    # First, determine indices for splitting
    test_split_index = X.shape[0] - int(np.round(X.shape[0] * test_size))
    val_split_index = test_split_index - int(np.round(X.shape[0] * validation_size))

    # Training set
    X_train = X[:val_split_index, :, :]
    y_train = y[:val_split_index, :]

    # Validation set
    X_val = X[val_split_index:test_split_index, :, :]
    y_val = y[val_split_index:test_split_index, :]

    # Test set
    X_test = X[test_split_index:, :, :]
    y_test = y[test_split_index:, :]

    print(f"Train shapes: X={X_train.shape}, y={y_train.shape}")
    print(f"Validation shapes: X={X_val.shape}, y={y_val.shape}")
    print(f"Test shapes: X={X_test.shape}, y={y_test.shape}")

    # Build models
    lstm_model = build_lstm_model((X_train.shape[1], X_train.shape[2]), units)
    lstm_model.summary()

    rnn_model = build_rnn_model((X_train.shape[1], X_train.shape[2]), units)
    rnn_model.summary()

    # Dictionary to store training times
    training_times = {}

    # Check if GPU is available
    if tf.config.list_physical_devices('GPU'):
        print("Using GPU for training")
        device = '/GPU:0'
    else:
        print("Using CPU for training")
        device = '/CPU:0'

    set_seeds(seed)
    # Train LSTM model with validation data and measure time
    with tf.device(device):
        print("\n--- Training LSTM Model ---")
        start_time = time.time()
        lstm_history = lstm_model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),  # Added validation data
            verbose=1
        )
        end_time = time.time()
        training_times['LSTM'] = end_time - start_time
        print(f"LSTM training time: {training_times['LSTM']:.2f} seconds")

        # Train RNN model with validation data and measure time
        set_seeds(seed)
        print("\n--- Training RNN Model ---")
        start_time = time.time()
        rnn_history = rnn_model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),  # Added validation data
            verbose=1
        )
        end_time = time.time()
        training_times['RNN'] = end_time - start_time
        print(f"RNN training time: {training_times['RNN']:.2f} seconds")

    # Plot training history comparison with validation losses
    plot_training_history(lstm_history, rnn_history)
    plot_individual_training_history(lstm_history, rnn_history)

    # Make predictions with both models
    lstm_pred_scaled = lstm_model.predict(X_test)
    lstm_pred = scaler.inverse_transform(lstm_pred_scaled)

    rnn_pred_scaled = rnn_model.predict(X_test)
    rnn_pred = scaler.inverse_transform(rnn_pred_scaled)

    # Ground truth (actual values)
    actual = scaler.inverse_transform(y_test)

    # Create DataFrames for predictions
    lstm_pred_df = pd.DataFrame(lstm_pred, columns=df.columns[:-1])
    lstm_pred_df.index = df.index[seq_length + test_split_index:]

    rnn_pred_df = pd.DataFrame(rnn_pred, columns=df.columns[:-1])
    rnn_pred_df.index = df.index[seq_length + test_split_index:]

    # Calculate and compare performance metrics
    lstm_metrics = calculate_metrics(actual[:, 0], lstm_pred[:, 0])  # Using only 'close' column
    rnn_metrics = calculate_metrics(actual[:, 0], rnn_pred[:, 0])    # Using only 'close' column

    compare_model_metrics(lstm_metrics, rnn_metrics)

    # Plot prediction comparisons
    plot_model_comparison(df, lstm_pred_df, rnn_pred_df)
    plot_error_comparison(df, lstm_pred_df, rnn_pred_df)
    plot_close_predictions_detail(df, lstm_pred_df, rnn_pred_df, window_size=30)

    # Generate forecasts with both models
    last_sequence = scaler.transform(lstm_pred_df.values[-seq_length:])
    future_start_date = '2025/3/4'

    print("\n--- Generating Forecasts ---")
    lstm_forecast = forecast_future(
        lstm_model,
        last_sequence,
        scaler,
        days_to_predict,
        seq_length,
        X_train.shape[2],
        future_start_date
    )
    lstm_forecast.columns = df.columns[:-1]

    rnn_forecast = forecast_future(
        rnn_model,
        last_sequence,
        scaler,
        days_to_predict,
        seq_length,
        X_train.shape[2],
        future_start_date
    )
    rnn_forecast.columns = df.columns[:-1]

    # Plot forecasts comparison
    plot_model_comparison(df, lstm_pred_df, rnn_pred_df, lstm_forecast, rnn_forecast)
    plot_forecast_comparison(lstm_forecast, rnn_forecast)

    # Print forecast results
    print("\n--- LSTM Forecast Results ---")
    print(lstm_forecast[['open', 'close']].head())
    print(f"LSTM forecast ending price: ${lstm_forecast['close'].iloc[-1]:.2f}")
    lstm_forecast_change = (lstm_forecast['close'].iloc[-1] - df['close'].iloc[-1]) / df['close'].iloc[-1] * 100
    print(f"LSTM percentage change from last known price: {lstm_forecast_change:.2f}%")

    print("\n--- RNN Forecast Results ---")
    print(rnn_forecast[['open', 'close']].head())
    print(f"RNN forecast ending price: ${rnn_forecast['close'].iloc[-1]:.2f}")
    rnn_forecast_change = (rnn_forecast['close'].iloc[-1] - df['close'].iloc[-1]) / df['close'].iloc[-1] * 100
    print(f"RNN percentage change from last known price: {rnn_forecast_change:.2f}%")

    # Final comparison summary
    print("\n--- Model Comparison Summary ---")
    print(f"LSTM training time: {training_times['LSTM']:.2f} seconds")
    print(f"RNN training time: {training_times['RNN']:.2f} seconds")
    print(f"Training time difference: {(training_times['LSTM'] - training_times['RNN']):.2f} seconds")

    # Validation performance
    lstm_val_loss = lstm_history.history['val_loss'][-1]
    rnn_val_loss = rnn_history.history['val_loss'][-1]
    print(f"Final LSTM validation loss: {lstm_val_loss:.6f}")
    print(f"Final RNN validation loss: {rnn_val_loss:.6f}")
    print(f"Validation loss difference: {(lstm_val_loss - rnn_val_loss):.6f}")

    forecast_diff = (lstm_forecast['close'].iloc[-1] - rnn_forecast['close'].iloc[-1])
    forecast_diff_pct = (forecast_diff / rnn_forecast['close'].iloc[-1]) * 100
    print(f"Final forecast difference: ${forecast_diff:.2f} ({forecast_diff_pct:.2f}%)")

    # Determine which model performed better overall
    mse_better = "LSTM" if lstm_metrics["MSE"] < rnn_metrics["MSE"] else "RNN"
    r2_better = "LSTM" if lstm_metrics["R²"] > rnn_metrics["R²"] else "RNN"
    val_better = "LSTM" if lstm_val_loss < rnn_val_loss else "RNN"

    print(f"Based on MSE, {mse_better} performed better")
    print(f"Based on R², {r2_better} performed better")
    print(f"Based on validation loss, {val_better} performed better")

    print("\nAnalysis complete")

In [ ]:
if  __name__ == "__main__":
  set_seeds(seed)
  main()

## Conclusion

This analysis demonstrates the application of recurrent neural networks to financial time series forecasting, specifically comparing LSTM and RNN architectures.

Key findings:
- LSTM vs RNN performance differences in terms of accuracy and training time
- The impact of model architecture on forecasting quality
- The challenges of multi-step recursive forecasting

Limitations and future work:
- Incorporating additional features (technical indicators, market sentiment)
- Testing more advanced architectures (Transformer models, attention mechanisms)
- Ensemble methods to combine multiple forecasts
- Hyperparameter optimization
- Exploring longer forecast horizons and their reliability

## Configuration Parameters

The analysis uses the following configuration parameters:

- **seq_length = 180**: Number of days in each input sequence
- **units = 20**: Number of neurons in the recurrent layers
- **batch_size = 32**: Number of samples per gradient update
- **epochs = 25**: Number of complete passes through the training dataset
- **days_to_predict = 30**: Forecast horizon in days
- **test_size = 0.15**: Proportion of data used for testing
- **validation_size = 0.15**: Proportion of data used for validation
- **n_steps_ma = 10**: Window size for moving average calculation

These parameters can be adjusted to experiment with different model configurations.